In [1]:
from tqdm import tqdm
from dlcliche.image import *

from fastai.vision import *
from torchvision import datasets, transforms
from torch import nn
import PIL
import time

In [2]:
import fastai
fastai.__version__

'1.0.60'

#田久保が作成したファイル
import get_embedding
import imscatter
import grad_cam
import body_feature_model
#クラスタリング
import image_clustering_2

In [3]:

FILE_NAME = "image/"
MODEL_FILE = "model/"

def prepare_full_MNIST_databunch(data_folder, tfms):
    """
    Prepare dataset as images under:
        data_folder/images/('train' or 'valid')/(class)
    where filenames are:
        img(class)_(count index).png
    """
    train_ds = datasets.MNIST(data_folder, train=True, download=True,
                          transform=transforms.Compose([
                              transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                          ]))
    valid_ds = datasets.MNIST(data_folder, train=False,
                              transform=transforms.Compose([
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                              ]))
    test_ds = datasets.MNIST(data_folder, train=False,
                              transform=transforms.Compose([
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                              ]))

    def have_already_been_done():
        return (data_folder/'images').is_dir()
    def build_images_folder(data_root, X, labels, dest_folder):
        images = data_folder/'images'
        for i, (x, y) in tqdm.tqdm(enumerate(zip(X, labels))):
            folder = images/dest_folder/f'{y}'
            ensure_folder(folder)
            x = x.numpy()
            image = np.stack([x for ch in range(3)], axis=-1)
            PIL.Image.fromarray(image).save(folder/f'img{y}_{i:06d}.jpg')

    if not have_already_been_done():
        build_images_folder(data_root=DATA, X=train_ds.train_data,
                            labels=train_ds.train_labels, dest_folder='train')
        build_images_folder(data_root=DATA, X=valid_ds.test_data, 
                            labels=valid_ds.test_labels, dest_folder='valid')
        build_images_folder(data_root=DATA, X=test_ds.test_data, 
                            labels=test_ds.test_labels, dest_folder='test')

    return ImageDataBunch.from_folder(data_folder/'images', ds_tfms=tfms,bs=16)


def body_feature_model(model,takubo=0):
    """
    Returns a model that output flattened features directly from CNN body.
    """
    if takubo == 0 :
        try:
            body, head = list(model.org_model.children()) # For XXNet defined in this notebook
        except:
            body, head = list(model.children()) # For original pytorch model
    
    return nn.Sequential(body, head[:-1])


def get_embeddings(embedding_model, data_loader, label_catcher=None, return_y=False):
    """
    Calculate embeddings for all samples in a data_loader.
    
    Args:
        label_catcher: LearnerCallback for keeping last batch labels.
        return_y: Also returns labels, for working with training set.
    """
    embs, ys = [], []
    xs = []
    for X, y in data_loader:
        # For each batch (X, y),
        #   Set labels (y) if label_catcher's there.
        if label_catcher:
            label_catcher.on_batch_begin(X, y, train=False)
        #   Get embeddings for this batch, store in embs.
        with torch.no_grad():
            # Note that model's output is not softmax'ed.
            out = embedding_model(X).cpu().detach().numpy()
            out = out.reshape((len(out), -1))
            embs.append(out)
        
        for i in X:
            xs.append(i.data.to("cuda").reshape(1, 3, 224, 224))
        y = y.detach().cpu().numpy()
        ys.append(y)
    # Putting all embeddings in shape (number of samples, length of one sample embeddings)
    embs = np.concatenate(embs) # Maybe in (10000, 10)
    ys = np.concatenate(ys,0)
    #xs = np.concatenate(xs) 
    if return_y:
        return embs, ys, xs
    return embs

def get_embeddings_one(embedding_model, image, label, return_y=False):
    """
    Calculate embeddings for all samples in a data_loader.
    
    Args:
        label_catcher: LearnerCallback for keeping last batch labels.
        return_y: Also returns labels, for working with training set.
    """
    embs = []
    with torch.no_grad():
            # Note that model's output is not softmax'ed.
        out = embedding_model(image).cpu().detach().numpy()
        out = out.reshape((len(out), -1))
        embs.append(out)        
    # Putting all embeddings in shape (number of samples, length of one sample embeddings)
    embs = np.concatenate(embs) # Maybe in (10000, 10)
    #xs = np.concatenate(xs) 
    if return_y:
        return embs, label, image
    return embs

In [4]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import axes3d
#%matplotlib notebook

def show_2D_tSNE(latent_vecs, target, title='t-SNE viz'):
    latent_vecs = latent_vecs
    latent_vecs_reduced = TSNE(n_components=2, random_state=0).fit_transform(latent_vecs)
    plt.scatter(latent_vecs_reduced[:, 0], latent_vecs_reduced[:, 1],
                c=target, cmap='jet')
    plt.colorbar()
    plt.savefig(FILE_NAME+title+".jpg")
    plt.show()


def show_3D_tSNE(latent_vecs, target, title='3D t-SNE viz'):
    latent_vecs = latent_vecs
    tsne = TSNE(n_components=3, random_state=0).fit_transform(latent_vecs)
    fig = plt.figure(figsize=(13,10))
    ax = fig.add_subplot(111, projection='3d')
    scatter = ax.scatter3D(tsne[:, 0], tsne[:, 1], tsne[:, 2], c=target, cmap='jet')
    ax.set_title(title)
    plt.colorbar(scatter)
    plt.savefig(FILE_NAME+title+".jpg")
    plt.show()


def show_as_PCA(latent_vecs, target, title='PCA viz'):
    latent_vecs = latent_vecs
    #latent_vecs_reduced = TSNE(n_components=2, random_state=0).fit_transform(latent_vecs)
    latent_vecs_reduced = PCA(n_components=2).fit_transform(latent_vecs)
    plt.scatter(latent_vecs_reduced[:, 0], latent_vecs_reduced[:, 1],
                c=target, cmap='jet')
    plt.colorbar()
    plt.show()

In [5]:
import requests
from bs4 import BeautifulSoup
import cv2
import pathlib
import numpy as np
from MulticoreTSNE import MulticoreTSNE as TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

%matplotlib inline

"""
t-sneで次元削除をしてそのグラフに画像をはる関数
image_list　画像ファイル名
latent_vecs　特徴抽出した画像の値
title　グラフ画像の保存名
"""

path_l=[]
for i in [4,0,6,2,1,3,5]:
    
    path = pathlib.Path('/home/cvmlab/Desktop/fastai/data/images/valid/'+str(i)+"/").glob('*.jpg')
    for p in path:
        path_l.append(p)

def imscatter(image_list,latent_vecs, title, ax=None, zoom=1):
    
    latent_vecs = latent_vecs
    kills_reduced = TSNE(n_components=2, random_state=0).fit_transform(latent_vecs)
    #tsne = TSNE(n_jobs=4, perplexity=20) # 20が一番いい感じでした
    #kills_reduced = tsne.fit_transform(latent_vecs)
    x,y= kills_reduced[:,0], kills_reduced[:,1]
    
    if ax is None:
        ax = plt.gca()
    im_list = [OffsetImage(plt.imread(str(p)), zoom=zoom) for p in image_list]
    x, y = np.atleast_1d(x, y)
    artists = []
    for x0, y0, im in zip(x, y, im_list):
        ab = AnnotationBbox(im, (x0, y0), xycoords='data', frameon=False)
        artists.append(ax.add_artist(ab))
    ax.update_datalim(np.column_stack([x, y]))
    ax.autoscale()
    return artists

    # perplexity: 20
    fig, ax = plt.subplots(figsize=(30,30))
    
    plt.savefig(FILE_NAME+title+".jpg")
    plt.show()


In [6]:
#MNISTがない場合は新しくつくってしまうので注意
DATA = Path('data')
data = prepare_full_MNIST_databunch(DATA, get_transforms(do_flip=False))
#image_path=[4,0,6,2,1,3,5]
image_path=[4,0,6,2,1,3,5]
data.valid_ds

LabelList (202 items)
x: ImageList
Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224),Image (3, 224, 224)
y: CategoryList
4,4,4,4,4
Path: data/images

In [7]:
try:
    from metrics import *
except:
    ! wget https://raw.githubusercontent.com/ronghuaiyang/arcface-pytorch/master/models/metrics.py
    from metrics import *

In [8]:
"""
ArcFaceで学習を行うファイル

"""
class LabelCatcher(LearnerCallback):
    last_labels = None

    def __init__(self, learn:Learner):
        super().__init__(learn)
        
    def on_batch_begin(self, last_input, last_target, train, **kwargs):
        LabelCatcher.last_labels = last_target
        return {'last_input': last_input, 'last_target': last_target} 


class XFaceNet(nn.Module):
    def __init__(self, org_model, data, xface_product=ArcMarginProduct, m=0.5):
        super().__init__()
        self.org_model = org_model
        self.feature_model = body_feature_model(org_model)
        self.metric_fc = xface_product(512, data.c, m=m).cuda()
    
    def forward(self, x):
        x = self.feature_model(x)
        x = self.metric_fc(x, LabelCatcher.last_labels)
        return x


def learner_ArcFace(train_data,train_num):
    learn = cnn_learner(train_data, models.resnet18, metrics=accuracy)
    learn.model = XFaceNet(learn.model, train_data, ArcMarginProduct, m=0.5)
    learn.callback_fns.append(partial(LabelCatcher))
    learn.fit(1)
    learn.unfreeze()
    learn.fit(train_num)
    return learn




In [12]:
#訓練回数
train_num = 50
learn = learner_ArcFace(data,train_num)
#learn.save('data_cnn_arcface_kimiaki')
#pred_class,pred_idx,outputs = learn.predict(img)  
#print(pred_class) 

epoch,train_loss,valid_loss,accuracy,time
0,14.825819,12.269404,0.059406,00:02


epoch,train_loss,valid_loss,accuracy,time
0,12.392128,9.734760,0.445545,00:02
1,9.689338,10.585092,0.519802,00:02
2,7.717578,9.960425,0.519802,00:02
3,6.270826,9.355492,0.574257,00:02
4,4.952044,8.778412,0.589109,00:02
5,4.379753,9.578675,0.564356,00:02
6,3.965181,10.300460,0.574257,00:02
7,3.274599,10.389733,0.579208,00:02
8,2.948407,7.895908,0.668317,00:02
9,2.995084,10.587428,0.589109,00:02


KeyboardInterrupt: 

In [ ]:
#特徴抽出（512次元）
embs = get_embeddings(body_feature_model(learn.model,takubo=0), data.valid_dl)
#t-sneで次元削除を行いグラフで可視化
show_2D_tSNE(embs, [int(y) for y in data.valid_ds.y], title='SphereFace (t-SNE)')
#t-sneで可視化（画像を貼り付けて）
fig, ax = plt.subplots(figsize=(30,30))
imscatter(path_l, embs, ax=ax, zoom=0.4,title="Simply resnet18")
#t-sneで可視化（3次元）
show_3D_tSNE(embs, [int(y) for y in data.valid_ds.y], title='Simply resnet18 (t-SNE3D)')


In [ ]:
import image_clustering_2
"""
田久保が書いたクラスタリング(k-means++)のファイルをここで実行している。
クラスタリングとARI（クラスタリングの評価）を出力する。
embs　cnnから特徴抽出を行った値(512次元)
n_clusters=3 クラスタ数←この数にクラス分けをしてくれる
image_file_temp='img_%s.jpg'　どのファイルを扱うかをしてする（ここではjpgファイル）
image_size=224　画像のサイズ
t_SNE_OK=False　クラスタリングの前にt-sneを使用するかどうか
"""
s=image_clustering_2.Image_Clustering(embs, n_clusters=7, image_file_temp='img_%s.jpg', image_size=224,t_SNE_OK=False,image_path=image_path)
s.main()

# grad_cam

In [ ]:
#grad_camについて



# hook for the gradients 
def gradient_torch_hook(self, grad_input, grad_output):
    return grad_input

def image_from_tensor(imagetensor):
    numpied = torch.squeeze(imagetensor)
    numpied = np.moveaxis(numpied.cpu().numpy(), 0 , -1)
    numpied = numpied - np.min(numpied)
    numpied = numpied/np.max(numpied)
    return numpied


#returnを書く
def Grad_cam(model,image,label,show_class_grad_cam,data,top50):
    #どこの層でするか
    #式で言うところのA
    target_layer = model.org_model[0][7][1]
    
    #time
    before = time.time()
    
    # hook for the feature maps
    #レイヤー(target_layer)の出力を格納するだけのHookを作成する関数
    fmap_hook = fastai.callbacks.hook_output(target_layer)
    
    gradient_hook = fastai.callbacks.Hook(target_layer, gradient_torch_hook, is_forward=False)
    
    model.eval()
    model.zero_grad()
    
   
    
    out = model.org_model(image)
    #クラスに合わせてゼロを作成している
    onehot = torch.zeros(data.c)

    #今調べたいクラスを記入するどこをみているのか
    class_grad_cam = show_class_grad_cam
    onehot[class_grad_cam] = 1.0
    out.backward(gradient=onehot.reshape(1, -1).cuda(), retain_graph=True)

    gradients = next(iter(gradient_hook.stored))

    gradient_linearization = gradients.cpu().numpy().sum((2, 3)).reshape(-1)
    
    #重みを平均化して、レイヤーのアウトプットに乗じる
    #weights = np.mean(gradient_linearization, axis=(0, 1))
    weights = np.mean(gradient_linearization)
    label = np.argsort(weights)
    
    
    fmaps = fmap_hook.stored.cpu().numpy()
    fmaps = fmaps.reshape(512, 7, 7)

    # relu on the heatmap
    #einsumはアインシュタンンの縮約記法法
    #gradient_linearization一元元
    #fmaps三元元
    #Σててる
    heatmap = np.maximum(0, np.einsum('i, ijk',gradient_linearization, fmaps))

    # we now upsample the heatmap so we can overlay it on our original image
    upsampled = scipy.ndimage.zoom(heatmap, 32)
    upsampled = (upsampled - np.min(upsampled))/(np.max(upsampled) - np.min(upsampled))
    
    return upsampled, gradient_linearization, time.time()-before
    
#画像を出力する。
"""
plt.imshow(image_from_tensor(image))
plt.imshow(upsampled, alpha=.8)
plt.gca().set_axis_off()
"""
    

In [ ]:
"""

"""
from sklearn.cluster import KMeans

def train_faster_gradcam(data,show_class_grad_cam,model,clusters=10,predict=False):
     #ここにかく
    """
    image=data.train_ds.x
    label = []
    for y in data.train_ds.y:
        label.append(y)
    """
    embs,y,x= get_embeddings(body_feature_model(model,takubo=0), data.train_dl,return_y=True)    
    # Arcfaceを削除
    model_embed = model.org_model[0][7][1]
    
    vector_anomaly = []
    vector_normal = []
    image_arr = []
    image_nomaly_arr = []
    label_arr = []
    label_nomaly_arr = []
    
    for i in range(len(x)):
        if show_class_grad_cam == int(y[i]):
            vector_anomaly.append(embs[i])
            image_arr.append(x[i])
            label_arr.append(y[i])
        else:
            vector_normal.append(embs[i])
            image_nomaly_arr.append(x[i])
            label_nomaly_arr.append(y[i])
            
    """
    for i in range(len(image)):
        if show_class_grad_cam == int(label[i]):
            vector_anomaly.append(embs[i])
            image_arr.append(image[i])
            label_arr.append(label[i])
    """
    
    # k-means
    kmeans = KMeans(n_clusters=clusters, random_state=0).fit(vector_anomaly)
    labels = kmeans.labels_

    # channel database
    channel_weight, channel_adress = [], []
    temp_weight = np.zeros((clusters, 512))# 480="block_16_expand_relu".output
    print("Making Database...")
    
    for i in range(len(labels)):
        # x_anomalyについて一個ずつ重みを加算していく
        _, weight,_ = Grad_cam(model=model,image=image_arr[i],label=label_arr[i],show_class_grad_cam=show_class_grad_cam,data=data,top50=False)
        #_, weight = Grad_cam(model=learn.model,image=img,label=label,show_class_grad_cam=4,data=data,top50=False)
        temp_weight[labels[i]] += weight #要確認show_class_grad_cam
        print(i+1,"/",len(labels))

    for i in range(clusters):
        number = np.where(labels == i, 1, 0) #クラスタ内の個数
        average_weight = temp_weight[i] / np.sum(number) #重みの平均
        weight_adress = np.argsort(average_weight)
        channel_adress.append(weight_adress[-50:])
        channel_weight.append(average_weight[weight_adress[-50:]])

    return model_embed, kmeans, np.array(channel_weight), np.array(channel_adress), vector_normal

"""
x:image

"""
def predict_faster_gradcam(model, kmeans, channel_weight, channel_adress,image,label):
    before = time.time()
    
    vector,channel_out,_ = get_embeddings_one(body_feature_model(model,takubo=0), image,label,return_y=True)
    
    target_layer = model.org_model[0][7][1]
    fmap_hook = fastai.callbacks.hook_output(target_layer)
    model.eval()
    model.zero_grad()
    
    channel_out = model.org_model(image)
    cluster_no = kmeans.predict(vector)
    
    fmaps = fmap_hook.stored.cpu().numpy()
    fmaps = fmaps.reshape(512, 7, 7)
    
    heatmap = np.maximum(0, np.einsum('i, ijk',channel_weight[cluster_no][0], fmaps[channel_adress[cluster_no][0],:,:]))

    # ヒートマップにして合成
    upsampled = scipy.ndimage.zoom(heatmap, 32)
    upsampled = (upsampled - np.min(upsampled))/(np.max(upsampled) - np.min(upsampled))
    """
    # レイヤーのアウトプットに乗じる
    cam = np.dot(channel_out[:,:,channel_adress[cluster_no][0]], channel_weight[cluster_no][0])

    cam = cv2.resize(cam, (x.shape[1], x.shape[0]), cv2.INTER_LINEAR)
    cam = np.maximum(cam, 0)
    cam = cam / cam.max()
    
    jetcam = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)  # モノクロ画像に疑似的に色をつける
    jetcam = cv2.cvtColor(jetcam, cv2.COLOR_BGR2RGB)  # 色をRGBに変換
    jetcam = (np.float32(jetcam) + x*255 / 2)   # もとの画像に合成
    """
   

    return upsampled,time.time()-before

In [ ]:
# compare grad and faster-grad
#imageとlabelをつくらなきゃいけない
#show_resultの変数のdataをわけてみるか
#返り値に気をつけてコードを編集する

def show_result(model, model_small, image,label,show_class_grad_cam, no, kmeans, channel_weight, channel_adress, top50=False):
    original, result_grad, result_faster, time0, time1 = [], [], [], [], []
    for i in range(5):
        original.append(image[no[i]]) 
        img0, _, time_0 = Grad_cam(model=model,image=image[no[i]],label=label[no[i]],show_class_grad_cam=show_class_grad_cam,data=data,top50=False)
        img1, time_1 = predict_faster_gradcam(model, kmeans, channel_weight, channel_adress,image[no[i]],label=label[no[i]])
        result_grad.append(img0)
        result_faster.append(img1)
        time0.append(time_0)
        time1.append(time_1)

    plt.figure(figsize=(15,10))
    for i in range(5):
        plt.subplot(3,5,i+1)
        plt.axis("off")
        if i == 0:
            plt.title("original")
        plt.imshow(image_from_tensor(original[i]))
    for i in range(5):
        plt.subplot(3,5,i+6)
        plt.axis("off")
        if i == 0:
            time_ = int(np.mean(time1)*1000)
            plt.title("Adapting-Grad-CAM \n(%d msec)" % time_)
        plt.imshow(image_from_tensor(original[i]))
        plt.imshow(result_faster[i], alpha=.8)
    for i in range(5):
        plt.subplot(3,5,i+11)
        plt.axis("off")
        if i == 0:
            time_ = int(np.mean(time0)*1000)
            plt.title("Grad-CAM \n(%d msec)" % time_)
        plt.imshow(image_from_tensor(original[i]))
        plt.imshow(result_grad[i], alpha=.8)
    plt.show()

In [ ]:
#grad-cam見たいラべル
"""
対話型タスクにするためには
show_class_grad_cam:grad-camに関してみたいラベルを記入する
target_label:画像のラベルを記入

通常のタスクにするためには
show_class_grad_camとtarget_labelを合わせるべき
"""
show_class_grad_cam = 3
target_label = 2

In [ ]:
model_grad, kmeans, channel_weight, channel_adress, vector_normal = train_faster_gradcam(data=data,show_class_grad_cam=show_class_grad_cam,model=learn.model, clusters=10)

In [ ]:
imgs = []
labels = []

for x,y in data.valid_dl:
    for i in x:
        imgs.append(i.data.to("cuda").reshape(1, 3, 224, 224))
    labels.append(y)
labels = np.concatenate(labels)

target_imgs=[]
target_imgs_label = []
for i in range(len(labels)):
    if int(labels[i]) == target_label:
        target_imgs.append(imgs[i])
        target_imgs_label.append(labels[i])
        
        
        
len(target_imgs)

In [ ]:
show_result(model=learn.model, model_small=model_grad,image=target_imgs,label=target_imgs_label,show_class_grad_cam=show_class_grad_cam, no=[1,2,3,4,5],kmeans=kmeans, channel_weight=channel_weight, channel_adress=channel_adress, top50=False)

# リアルタイム

In [ ]:
import cv2
import time
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.externals import joblib



channel_weight = channel_weight
channel_adress = channel_adress
#vector_normalがvector_paに変更する。
vector_normal = vector_normal
kmeans = kmeans
model = learn.model

#x1とx2のcos類似度をはかる。そのスコアを出力する。
def get_score_arc(vector_normal, test):
    # cosine similarity
    cos_similarity = cosine_similarity(test, vector_normal) # shape(len(test), len(train))

    return np.max(cos_similarity)

#x1とx2のcos類似度をはかる。
def cosine_similarity(x1, x2):
    #np.newaxisは計算するために次元をうまくやってくれるやつ
    if x1.ndim == 1:
        x1 = x1[np.newaxis]
    if x2.ndim == 1:
        x2 = x2[np.newaxis]
    #ベクトルの大きさ（ノルム）を計算する手法
    x1_norm = np.linalg.norm(x1, axis=1)
    x2_norm = np.linalg.norm(x2, axis=1)
    cosine_sim = np.dot(x1, x2.T)/(x1_norm*x2_norm+1e-10)
    return cosine_sim

#ここで推論をしている。ヒートマップを計算する関数
#ここは後で書き直します
def predict_faster_gradcam(channel, vector, img, kmeans, channel_weight, channel_adress):
    channel_out = channel[0]
    
    # k-means and heat_map
    cluster_no = kmeans.predict(vector)
    fmaps = fmap_hook.stored.cpu().numpy()
    fmaps = fmaps.reshape(512, 7, 7)
    
    heatmap = np.maximum(0, np.einsum('i, ijk',channel_weight[cluster_no][0], fmaps[channel_adress[cluster_no][0],:,:]))

    # ヒートマップにして合成
    upsampled = scipy.ndimage.zoom(heatmap, 32)
    upsampled = (upsampled - np.min(upsampled))/(np.max(upsampled) - np.min(upsampled))
    return upsampled

def get_x_y_limit(heatmap, thresh):
    #条件に応じた処理を行う
    map_ = np.where(heatmap>thresh)
    x_max = np.max(map_[1])
    x_min = np.min(map_[1])
    y_max = np.max(map_[0])
    y_min = np.min(map_[0])

    x_max = int(x_max)
    x_min = int(x_min)
    y_max = int(y_max)
    y_min = int(y_min)
    return x_min, y_min, x_max, y_max

#bounding_boxを描く関数
def bounding_box(img, x_min, y_min, x_max, y_max):
    img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 5)
    return img

def main():
    camera_width =  352
    camera_height = 288
    input_size = 96
    hand_thresh = 0.25
    OD_thresh = 0.8
    fps = ""
    message1 = "Push [q] to quit."
    message2 = "Push [s] to change mode."
    hand = ""
    elapsedTime = 0
    like_OD = False# like object detection

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FPS, 8)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, camera_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, camera_height)

    time.sleep(1)

    while cap.isOpened():
        t1 = time.time()

        ret, image = cap.read()
        image = image[:,32:320]
        if not ret:
            break

        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (input_size, input_size))
        #ここをかえる
        channel_out = model.org_model(img)
        test_vector = get_embeddings_one(body_feature_model(model,takubo=0), image=img, label=0, return_y=False)
        score = get_score_arc(vector_pa, test_vector)

        if score < hand_thresh: # hand is gu
            hand = "gu"
            color = (255, 0, 0)
            heatmap = predict_faster_gradcam(channel_out, test_vector, image, kmeans, channel_weight, channel_adress)
            if like_OD:
                x_min, y_min, x_max, y_max = get_x_y_limit(heatmap, OD_thresh)
                result = bounding_box(image, x_min, y_min, x_max, y_max)
            else:
                heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
                image = np.copy(cv2.addWeighted(heatmap, 0.5, image, 0.5, 2.2))

        else:# hand is pa
            hand = "pa"
            color = (0, 0, 255)

        # message
        cv2.putText(image, "{0} {1:.1f} Score".format(hand, score),(camera_width - 290, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 1, cv2.LINE_AA)
        cv2.putText(image, message1, (camera_width - 285, 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, message2, (camera_width - 285, 35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, fps, (camera_width - 175, 110), cv2.FONT_HERSHEY_SIMPLEX, 1, ( 255, 0 ,0), 1, cv2.LINE_AA)

        cv2.imshow("Result", image)

        # FPS
        elapsedTime = time.time() - t1
        fps = "{:.0f} FPS".format(1/elapsedTime)

        # quit or change mode
        key = cv2.waitKey(1)&0xFF
        if key == ord("q"):
            break
        if key == ord("s"):
            if like_OD == True:
                like_OD = False
            else:
                like_OD = True

    cv2.destroyAllWindows()


In [ ]:
main()

In [ ]:
import cv2
cap = cv2.VideoCapture(0)
while True:
    ret, img = cap.read()
    cv2.imshow('video image', img)#'video image'はカメラのウィンドウの名前
    key = cv2.waitKey(10)
    if key == 27:  # ESCキーで終了
        break
cap.release()
cv2.destroyAllWindows()